# Loading pre-defined graphs and generating synthetic graphs using NetworkX

### Setup

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Data Loading

For this Colab we will be using [NetworkX](https://networkx.github.io), a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks ([tutorial](https://networkx.org/documentation/stable/tutorial.html)).

The dataset we will analyze is a snapshot of the Epinions.com network. Nodes represent people and directed edges represent a trusting relationship. [[More Info]](https://snap.stanford.edu/data/soc-Epinions1.html)

In [6]:
import networkx as nx

G = nx.read_edgelist('soc-Epinions1.txt.gz', create_using=nx.DiGraph)

#### Function to print graph information

In [7]:
def print_graph_info(G, directed=True):
  print("Number of nodes:", len(G.nodes))
  print("Number of edges:", len(G.edges))
  if directed:
    print("Average in-degree:", sum(dict(G.in_degree).values()) / len(G.nodes))
    print("Average out-degree:", sum(dict(G.out_degree).values()) / len(G.nodes))
  else:
    print("Average degree:", sum(dict(G.degree).values()) / len(G.nodes))

In [8]:
print_graph_info(G, True)

Number of nodes: 75879
Number of edges: 508837
Average in-degree: 6.7059001831863885
Average out-degree: 6.7059001831863885


### Your Task

To begin with, let's simplify our analysis by ignoring the dangling nodes and the disconnected components in the original graph.

Use NetworkX to identify the **largest** weakly connected component in the ```G``` graph.  From now on, use this connected component for all the following tasks.

Print its information.

In [9]:
largest_cc = max(nx.weakly_connected_components(G), key=len)
CSG = G.subgraph(largest_cc)
print_graph_info(CSG, True)

Number of nodes: 75877
Number of edges: 508836
Average in-degree: 6.706063761087022
Average out-degree: 6.706063761087022


Compute the PageRank vector, using the default parameters in NetworkX: [https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html#networkx.algorithms.link_analysis.pagerank_alg.pageranky](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html#networkx.algorithms.link_analysis.pagerank_alg.pagerank)

In [18]:
pr_CSG = nx.pagerank(CSG)

In 1999, Barabási and Albert proposed an elegant mathematical model which can generate graphs with topological properties similar to the Web Graph (also called Scale-free Networks).

If you complete the steps below, you should obtain some empirical evidence that the Random Graph model is inferior compared to the Barabási–Albert model when it comes to generating a graph resembling the World Wide Web!

As such, we will use two different graph generator methods, and then we will test how well they approximate the Web Graph structure by means of comparing the respective PageRank vectors. [[NetworkX Graph generators]](https://networkx.github.io/documentation/stable/reference/generators.html#)

Using the parameters ```seed = 1``` and ```directed=False``` where applicable, generate:


1.   a random graph (with the fast method), setting ```n``` equal to the number of nodes in the original connected component, and ```p = 0.00008```
2.   a Barabasi-Albert graph (with the standard method), setting ```n``` equal to the number of nodes in the original connected component, and finding the right ***integer*** value for ```m``` such as the resulting number of edges **approximates by excess** the number of edges in the original connected component

and compute the PageRank vectors for both graphs. Print generated graph's information, if needed.


In [16]:
FRG = nx.fast_gnp_random_graph(75877, 0.0002, seed=1, directed=False)
print_graph_info(FRG, False)
pr_FRG = nx.pagerank(FRG)
print("Node with largest PageRank value:", max(pr_FRG, key=pr_FRG.get))

Number of nodes: 75877
Number of edges: 575521
Average degree: 15.169840663178565
Node with largest PageRank value: 57235


In [12]:
BAG = nx.barabasi_albert_graph(75877, 7, seed=1, initial_graph=None)
print_graph_info(BAG, False)
pr_BAG = nx.pagerank(BAG)
print("Largest PageRank value:", max(pr_BAG.values()))

Number of nodes: 75877
Number of edges: 531090
Average degree: 13.998708436021456
Largest PageRank value: 0.0009930323919684033


Compare the PageRank vectors obtained on the generated graphs with the PageRank vector you computed on the original connected component.
**Sort** the components of each vector by value, and use cosine similarity as similarity measure.

In [17]:
pr_CSG_sorted = np.sort(list(pr_CSG.values()))
pr_FRG_sorted = np.sort(list(pr_FRG.values()))
pr_BAG_sorted = np.sort(list(pr_BAG.values()))

cos_FRG = np.dot(pr_CSG_sorted,pr_FRG_sorted)/(np.linalg.norm(pr_CSG_sorted)*np.linalg.norm(pr_FRG_sorted))
print("Cosine Similarity of Random Graph:", cos_FRG)

cos_BAG = np.dot(pr_CSG_sorted,pr_BAG_sorted)/(np.linalg.norm(pr_CSG_sorted)*np.linalg.norm(pr_BAG_sorted))
print("Cosine Similarity of Barabasi-Albert Graph:", cos_BAG)

Cosine Similarity of Random Graph: 0.308282916404342
Cosine Similarity of Barabasi-Albert Graph: 0.8807978419601572
